#### Yelp Fusion API      
1. https://github.com/Yelp/yelp-fusion/tree/master/fusion/python     
2. $pip install -r requirements.txt
3. By default, you’ll receive 25,000 calls per day. If you expect to hit this rate limit, please contact api@yelp.com with links to or screenshots of your app or website.      
https://www.yelp.com/developers/faq

### confidentials
App ID: ezS3PAsVkMfPq5bqVvAXUg      
App Secret: zZIaiWTiyuUvuXM7EYMUvxt52jdKVftFl5TjIR4ByCt2rwzR9txyzV0EkkmHlJJt

In [1]:
# OAuth credential placeholders that must be filled in by users.
# You can find them on
# https://www.yelp.com/developers/v3/manage_app
CLIENT_ID = 'ezS3PAsVkMfPq5bqVvAXUg'
CLIENT_SECRET = 'zZIaiWTiyuUvuXM7EYMUvxt52jdKVftFl5TjIR4ByCt2rwzR9txyzV0EkkmHlJJt'

In [2]:
from __future__ import print_function

import json
import pprint
import requests
import sys
import urllib

from urllib2 import HTTPError
from urllib import quote
from urllib import urlencode

In [3]:
# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
TOKEN_PATH = '/oauth2/token'
GRANT_TYPE = 'client_credentials'

### Yelp functions

In [4]:
def obtain_bearer_token(host, path):
    """Given a bearer token, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        str: OAuth bearer token, obtained using client_id and client_secret.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    assert CLIENT_ID, "Please supply your client_id."
    assert CLIENT_SECRET, "Please supply your client_secret."
    data = urlencode({
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': GRANT_TYPE,
    })
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }
    response = requests.request('POST', url, data=data, headers=headers)
    bearer_token = response.json()['access_token']
    return bearer_token


def request(host, path, bearer_token, url_params=None):
    """Given a bearer token, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        bearer_token (str): OAuth bearer token, obtained using client_id and client_secret.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % bearer_token,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(bearer_token, term, location):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, bearer_token, url_params=url_params)


def get_business(bearer_token, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, bearer_token)


def query_api(term, location):
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    bearer_token = obtain_bearer_token(API_HOST, TOKEN_PATH)

    response = search(bearer_token, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(bearer_token, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)

### Test case

In [7]:
# Defaults for our simple example.
term = 'doctor'
location = 'San Francisco, CA'
SEARCH_LIMIT = 50

In [8]:
try:
    query_api(term, location)
except HTTPError as error:
    sys.exit(
        'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
            error.code,
            error.url,
            error.read(),
        )
    )

Querying https://api.yelp.com/v3/businesses/search ...
50 businesses found, querying business info for the top result "david-shu-md-san-francisco" ...
Querying https://api.yelp.com/v3/businesses/david-shu-md-san-francisco ...
Result for business "david-shu-md-san-francisco" found:
{ u'categories': [{ u'alias': u'internalmed', u'title': u'Internal Medicine'}],
  u'coordinates': { u'latitude': 37.73165, u'longitude': -122.47418},
  u'display_phone': u'(415) 452-1200',
  u'id': u'david-shu-md-san-francisco',
  u'image_url': u'https://s3-media3.fl.yelpcdn.com/bphoto/nWVgX_p9VB0S5kK8u3ZCXQ/o.jpg',
  u'is_claimed': False,
  u'is_closed': False,
  u'location': { u'address1': u'2645 Ocean Ave',
                 u'address2': u'Ste 303',
                 u'address3': u'',
                 u'city': u'San Francisco',
                 u'country': u'US',
                 u'cross_streets': u'Lagunitas Dr & Woodacre Dr',
                 u'display_address': [ u'2645 Ocean Ave',
                       